# This project has purpose to get better rainfall estimates from infrared satellite datas in Bandung Basin, West Java. 

The predictor data were used from Himawari-8 satellite which is 9 infrared bands and their 36 combination from all infrared bands. Then the
predictive data used from the estimated rainfall from the GPM DPR KuPR. 

There are different spatial resolution data between Himawari-8 and GPM DPR KuPR, so I use interpolation first using nearest neighbour for both of the data to ~5 km.

Data sampling was carried out by extending data collection to one island of Java with the assumption of similar atmospheric conditions to overcome data limitations in the Bandung Basin. The algorithm used at the random forest model creation stage is to classify the rain area, classify the type of rain, and regress to get the rain value. 

# Import library

In [136]:
import numpy as np
import pandas as pd

#split data
from sklearn.model_selection import train_test_split

#ML_model_random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

#metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix,classification_report

# Open Data

disini kalau bisa diplot semuanya dulu, ntar diseleksi disini yang paling berpengaruh langsung. 

In [39]:
d = pd.read_csv('C:/Users/ajiwi/Documents/kodingan_TA/df_hmw-gpm_final.csv')
d.shape

(218199, 51)

#### The infrared variable used in this project have selected into 10 combination bands for the best brightness temperature and brightness temperature difference to predict precipitation based on calculation and function of the atmospheric factors. 

In [40]:
d1 = d[['10.4-11.2', '8.6-10.4','8.6-12.4','10.4-12.4', '6.2-6.9', '7.3-10.4','7.3-9.6','6.2-9.6','7.3-12.4','7.3-13.3', 'prec']]
d1.shape

(218199, 11)

# check & dropn null

In [41]:
null_columns = d1.columns[d1.isnull().any()]
d1[null_columns].isnull().sum()
d1.dropna()

,10.4-11.2,8.6-10.4,8.6-12.4,10.4-12.4,6.2-6.9,7.3-10.4,7.3-9.6,6.2-9.6,7.3-12.4,7.3-13.3,prec
0,1.415924,-1.579254,4.410126,5.989380,-14.492111,-30.503082,-7.440216,-32.296463,-24.513702,-7.445953,0.0
1,2.101044,-1.394836,5.635712,7.030548,-13.890808,-30.046143,-8.120178,-31.930740,-23.015594,-6.397217,0.0
2,2.362030,-1.263184,5.976440,7.239624,-13.890808,-29.846985,-8.060333,-32.249588,-22.607361,-5.877136,0.0
3,1.645813,-1.026703,4.908417,5.935120,-13.846985,-27.729675,-6.625458,-30.563110,-21.794556,-5.960022,0.0
4,1.759857,-1.309326,5.220673,6.529999,-14.005066,-29.430481,-7.646118,-31.733047,-22.900482,-6.480743,0.0
...,...,...,...,...,...,...,...,...,...,...,...
218194,0.927582,-3.008270,2.810150,5.818420,-11.431274,-41.570129,-16.384644,-40.200821,-35.751709,-17.609344,0.0
218195,0.386780,-3.167969,0.965973,4.133942,-11.499985,-37.467438,-13.004333,-37.362335,-33.333496,-16.931000,0.0
218196,0.285553,-2.970642,0.976410,3.947052,-11.446976,-36.832855,-13.050842,-36.972015,-32.885803,-16.601929,0.0
218197,0.252625,-3.194489,0.676117,3.870605,-11.274170,-36.843414,-12.877533,-36.466141,-32.972809,-16.528748,0.0


In [42]:
d1.shape

(218199, 11)

# Data Preparation

In [43]:
nlen2 = 218199
labels2 = np.array(list(range(nlen2)))
ndarr2 = np.array([0, 10, 20, 30, 40, 50])
rnum = labels2[d1['prec'] > 0. ]
cnum = labels2[d1['prec'] == 0.]
rcnt = len(rnum)
ccnt = len(cnum)
print('rnum(rain)   =',rcnt)
print('cnum(no_rain)=',ccnt)

rnum(rain)   = 17563
cnum(no_rain)= 200636


In [44]:
hnum = labels2[ d1['prec'] > 1.8 ]
snum = labels2[ (d1['prec'] > 0.) & (d1['prec'] <= 1.8) ]

hcnt = len(hnum)
scnt = len(snum)

print('hcnt(strong)=', hcnt)
print('scnt(weak)=', scnt)

hcnt(strong)= 5935
scnt(weak)= 11628


In [45]:
hcnt

5935

In [46]:
ratioR = 1.5
probR = ((rcnt*ratioR)/ccnt)

In [47]:
from sklearn.model_selection import train_test_split

cnum_train, cnum_test, = train_test_split(cnum, train_size=probR, random_state = 40)

In [48]:
dfc = d1.loc[cnum_train,:]
dfr = d1.loc[rnum,:]
dfc['classification']= 0
dfr['classification']= 1
df = pd.concat((dfc, dfr), axis=0)

In [49]:
df.iloc[:,:]

,10.4-11.2,8.6-10.4,8.6-12.4,10.4-12.4,6.2-6.9,7.3-10.4,7.3-9.6,6.2-9.6,7.3-12.4,7.3-13.3,prec,classification
134118,1.807114,0.679688,4.131042,3.451355,-2.807220,-3.845245,-5.282028,-10.161392,-0.393890,1.564331,0.000000,0
15319,0.161926,-3.117493,-1.080688,2.036804,-8.686646,-26.262909,-6.863220,-25.070496,-24.226105,-13.248596,0.000000,0
53314,1.090668,-2.248199,2.787079,5.035278,-9.555450,-30.066437,-7.578064,-25.941254,-25.031158,-12.319885,0.000000,0
162897,1.723541,-1.921539,1.570129,3.491669,-7.415115,-14.682205,-3.898163,-18.152283,-11.190536,-2.674103,0.000000,0
180800,0.946899,-2.485626,2.421722,4.907349,-10.247162,-32.087677,-7.910004,-24.451736,-27.180328,-12.388245,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
210413,0.570099,-1.735931,0.106934,1.842865,-10.334091,-16.867737,0.266418,-18.301437,-15.024872,-6.422791,1.378972,1
210452,0.005280,-2.335663,-1.050751,1.284912,-10.387131,-17.246307,1.269440,-17.427872,-15.961395,-7.185486,0.447752,1
210887,0.232513,-1.964905,-0.658936,1.305969,-10.849243,-17.947601,0.041321,-19.241287,-16.641632,-7.743774,0.278448,1
211186,0.041534,-2.251801,-0.131805,2.119995,-9.317444,-27.278870,-5.652435,-22.434219,-25.158875,-12.217743,0.259473,1


In [50]:
bands_T = df.iloc[:,0:10]
clf_T   = df.iloc[:,11]

# MODELING

### Classification Rain Area ( rain and no rain)

In [95]:
#separate to train & test data
X_train, X_val, y_train, y_val = train_test_split(bands_T, clf_T, random_state = 40, test_size = 0.2)

In [96]:
# define model
model_rainarea = RandomForestClassifier(n_estimators=500, oob_score= True)

In [97]:
#making model
model_rainarea.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, oob_score=True)

In [98]:
predict_rainarea = model_rainarea.predict(X_val)
conf_matrix = confusion_matrix(predict_rainarea, y_val)
print(conf_matrix)

class_report = classification_report(predict_rainarea, y_val)
print(class_report)

[[705 441]
 [484 744]]
              precision    recall  f1-score   support

           0       0.59      0.62      0.60      1146
           1       0.63      0.61      0.62      1228

    accuracy                           0.61      2374
   macro avg       0.61      0.61      0.61      2374
weighted avg       0.61      0.61      0.61      2374



=======================================================================================

In [54]:
ratioT =  1
probT = ((hcnt*ratioT)/scnt)
from sklearn.model_selection import train_test_split

snum_train, snum_test, = train_test_split(snum, train_size=probT, random_state = 40)

In [55]:
snum_test.sort()
snum_train.sort()

In [56]:
dfs = d1.loc[snum_train,:]
dfh = d1.loc[hnum,:]

dfs['class_T']= 0
dfh['class_T']= 1
df = pd.concat((dfs, dfh), axis=0)

In [57]:
bands_T = df.iloc[:,0:10]
clf_T   = df.iloc[:,11]

In [58]:
bands_T

,10.4-11.2,8.6-10.4,8.6-12.4,10.4-12.4,6.2-6.9,7.3-10.4,7.3-9.6,6.2-9.6,7.3-12.4,7.3-13.3
2185,1.693268,-0.234344,2.807648,3.041992,-10.066010,-11.341507,-0.093536,-18.093094,-8.299515,-1.406662
2335,1.763977,-0.443878,2.337662,2.781540,-9.351761,-10.337128,-0.005493,-17.050140,-7.555588,-1.023712
2362,1.132767,-0.458939,0.963654,1.422592,-6.950455,-7.259460,1.038467,-10.437454,-5.836868,-1.945023
2363,0.862427,-1.575211,0.544067,2.119278,-8.651108,-10.312042,0.125534,-14.610840,-8.192764,-2.451157
2364,0.726440,-1.129974,0.822052,1.952026,-11.704132,-14.559845,0.149261,-21.682678,-12.607819,-5.170166
...,...,...,...,...,...,...,...,...,...,...
201168,0.642334,-2.973602,0.721191,3.694794,-8.273315,-31.944519,-11.209625,-27.895370,-28.249725,-15.600494
201174,0.872345,-2.341125,1.732117,4.073242,-8.157089,-30.892273,-10.489777,-26.959518,-26.819031,-14.610931
201176,0.664581,-2.849487,1.487976,4.337463,-8.316498,-34.420654,-13.204926,-29.957397,-30.083191,-16.242432
206851,0.076019,-1.991425,1.083954,3.075378,-11.510803,-27.880371,-4.658875,-24.471039,-24.804993,-9.667847


### Classification Rain Type (strong or Weak rain)

In [59]:
#separate to train & test data
X_train_T, X_val_T, y_train_T, y_val_T = train_test_split(bands_T, clf_T, random_state = 92, test_size = 0.2)

In [60]:
# define model
model_Type = RandomForestClassifier(n_estimators=500, oob_score= True)

In [61]:
#making model
model_Type.fit(X_train_T, y_train_T)

RandomForestClassifier(n_estimators=500, oob_score=True)

In [101]:
predict_raintype = model_Type.predict(X_val_T)
conf_matrix_T = confusion_matrix(predict_raintype, y_val_T)
print(conf_matrix_T)

class_report_T = classification_report(predict_raintype, y_val_T)
print(class_report_T)

[[742 453]
 [437 742]]
              precision    recall  f1-score   support

           0       0.63      0.62      0.63      1195
           1       0.62      0.63      0.63      1179

    accuracy                           0.63      2374
   macro avg       0.63      0.63      0.63      2374
weighted avg       0.63      0.63      0.63      2374



=================================================================

=================================================================

#### Regression to get weak Rain value

In [108]:
band_weak = dfs.iloc[:,0:10] 
rain_weak = dfs.iloc[:,10]

In [109]:
#separate to train & test data
X_train_Rw, X_val_Rw, y_train_Rw, y_val_Rw = train_test_split(band_weak, rain_weak, random_state = 40, test_size = 0.2)

In [110]:
# define model
model_Rw = RandomForestRegressor(oob_score = True)

In [111]:
#making model
model_Rw.fit(X_train_Rw, y_train_Rw)

RandomForestRegressor(oob_score=True)

In [116]:
predict_Rw = model_Rw.predict(X_val_Rw)

In [119]:
MAE = mean_absolute_error(predict_Rw, y_val_Rw)
print(MAE)

0.36019958485290726


==========================================================
#### Regression to get Strong Rain value

In [120]:
band_strong = dfh.iloc[:,0:10] 
rain_strong = dfh.iloc[:,10]

In [121]:
#separate to train & test data
X_train_Rs, X_val_Rs, y_train_Rs, y_val_Rs = train_test_split(band_strong, rain_strong, random_state = 40, test_size = 0.2)

In [122]:
# define model
model_Rs = RandomForestRegressor(oob_score = True)

In [123]:
#making model
model_Rs.fit(X_train_Rs, y_train_Rs)

RandomForestRegressor(oob_score=True)

In [124]:
predict_Rs = model_Rs.predict(X_val_Rs)

In [126]:
MAE = mean_absolute_error(predict_Rs, y_val_Rs)
print(MAE)

5.3861427759831955


# Nama model


In [71]:
print (" model_rainarea \n, model_Type \n, model_Rw \n ,model_Rs")

 model_rainarea 
, model_Type 
, model_Rw 
 ,model_Rs


# Save the Model

In [133]:
import joblib 

In [135]:
# Save the model as a pickle in a file 
joblib.dump(model_rainarea, 'M_RArea.pkl')
joblib.dump(model_Type, 'M_RType.pkl')
joblib.dump(model_Rw, 'M_Rweak.pkl')
joblib.dump(model_Rs, 'M_Rstrong.pkl')

['M_Rstrong.pkl']